In [4]:
from glob import glob
import re


In [2]:
def clean(fn):
    with open(fn,'r') as infile:
        md = infile.read()
    
    # remove any indexes from last time
    
    old_indexs = re.findall('```\{index\}.*?```', md, re.DOTALL)
    for old_index in old_indexs:
        md = md.replace(old_index,'')
    
    # remove windows smartquotes:
    table = {0x201c: '"', 0x201d: '"',
        0x2018: "'", 0x2019: "'"}
    md = md.translate(table)
    
    meta_text = md.split('-->')[0].split('<!--')[-1].splitlines()
    meta_d = {'fn': fn}
    for row in meta_text:
        if ':' in row:
            p = row.split(':')
            
            k = p[0]
            v = ':'.join(p[1:])
            
            k,v = k.strip(), v.strip()
            meta_d[k] = v
    if 'author' not in meta_d:
        meta_d['author'] = 'W.E.B. Du Bois'
    if 'journal' not in meta_d:
        meta_d['journal'] = 'The Crisis'    

    
    meta_d["pages"] = meta_d["pages"].replace('--','&ndash;')
    meta_d["pages"] = meta_d["pages"].replace('-','&ndash;')
    
    title_text = f'''# {meta_d["title"]} ({meta_d["year"]})'''
    new_cite_line = f'''*Citation:* {meta_d["author"]}. {meta_d["year"]}. "{meta_d["title"]}." *{meta_d["journal"]}*. {meta_d["volume"]}({meta_d["issue"]}):{meta_d["pages"]}.'''
    new_cite_line = new_cite_line.replace('.. 1', '. 1')
    new_cite_line = new_cite_line.replace(''''."''','''.'"''' )
    
    for row in md.splitlines():
        if row[:2] == '# ':
            old_title_line = row
        if '*Citation' in row:
            old_cite_line = row
            
    #update
    try:
        md = md.replace(old_title_line, title_text)
    except UnboundLocalError:
        meta_end = md.find('-->')+ len('-->')
        md = md[:meta_end] + f'\n{title_text}\n' + md[meta_end:]
                           
        
    try:
        md = md.replace(old_cite_line, new_cite_line)
    except UnboundLocalError:
        md = md + f'\n{new_cite_line}'
    
    while '\n\n\n' in md:
        md = md.replace('\n\n\n', '\n\n')

    # Add index info
    
    index_directive = ''
    try:   
        index_text = meta_d['index']
        for index in index_text.split('| '):
            index_directive =  index_directive + "```{index} " + index + '\n```\n'
    except KeyError:
        pass
    
    '''if len(index_directive) > 0:
        md = md + index_directive
    '''    
        
    md = '\n'.join([blockquote_fix(row) for row in md.splitlines()])
    
    
    with open(fn,'w') as outfile:
        outfile.write(md)

# Attempt to fix quotes in block quotes"
def blockquote_fix(row):
    old_row = row
    if len(row)<1:
        return row
    row = row.replace('>"',"> ")
    row = row.replace('> "',"> ")
    row = row.replace('>  "',"> ")
    row = row.replace('>   "',"> ")
    
    if row[0]==">" and row[-1]=='"':
        row = row[:-1]
    if row[0]==">" and row[-2:]=='" ':
        row = row[:-2]
        
    return row


        

In [3]:
fns = glob('Volumes/**/*.md', recursive=True)
#fns = glob('Volumes/16/01/*.md', recursive=True)

fns = sorted(fns)

for fn in fns:
    clean(fn)

In [7]:
a = '''> Dog" '''

blockquote_fix(a)

'> Dog'